In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

2024-11-28 20:29:18.103876: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-28 20:29:18.112068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-28 20:29:18.120328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-28 20:29:18.122684: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 20:29:18.129712: I tensorflow/core/platform/cpu_feature_guar

In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpuid = 0 #int(args.gpu_id)                                                                                                                           
if gpus:
  # Restrict TensorFlow to only allocate X GB of memory on the first GPU                                                                              
  try:
    tf.config.set_visible_devices(gpus[gpuid], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[gpuid], True)
    '''
    tf.config.set_logical_device_configuration(
        gpus[gpuid],
        [tf.config.LogicalDeviceConfiguration(memory_limit=12000)])
    '''
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized                                                                                   
    print(e)

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1732850960.843526  381004 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850960.868927  381004 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850960.871135  381004 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732850960.873799  381004 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
import pandas as pd 
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    dataset = pd.read_csv("diabetes.csv",skiprows=1)
    dataset.head()

In [4]:
batchsize = 32
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    x = dataset.iloc[:,0:8]
    y = dataset.iloc[:,8:]
    y.head()

In [5]:

# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_shape=(8,), activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# load dataset
#dataset = np.loadtxt("diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
# create model
model = KerasClassifier(model=create_model,verbose=1)
# define the grid search parameters
#batch_size = [60,100]
epochs = [10, 50, 100]
param_grid = dict(epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(x, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/user/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


I0000 00:00:1732850970.634995  381147 service.cc:146] XLA service 0x7bf7c40076d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732850970.635016  381147 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2024-11-28 20:29:30.644217: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-28 20:29:30.699915: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3512 - loss: 45.1752 
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.3512 - loss: 34.8501
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - accuracy: 0.3516 - loss: 24.6355
Epoch 4/10


I0000 00:00:1732850971.025417  381147 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5956 - loss: 2.9728
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 319us/step - accuracy: 0.6176 - loss: 2.2267
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - accuracy: 0.6131 - loss: 1.9202
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.5984 - loss: 1.6367
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - accuracy: 0.5901 - loss: 1.4712
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 378us/step - accuracy: 0.6076 - loss: 1.3924
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - accuracy: 0.6066 - loss: 1.3280
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.5953 - loss: 1.2706
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5890 - loss: 1.2242
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step - accuracy: 0.5981 - loss: 1.1863
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - accuracy: 0.5891 - loss: 1.1571
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step -

/home/user/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.3780 - loss: 24.6250
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - accuracy: 0.3888 - loss: 17.4417
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 372us/step - accuracy: 0.4023 - loss: 12.5917
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.4393 - loss: 9.0861
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 347us/step - accuracy: 0.4935 - loss: 6.0316
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 354us/step - accuracy: 0.5578 - loss: 4.1101
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step - accuracy: 0.5343 - loss: 3.5106
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - accuracy: 0.4787 - loss: 2.8834
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.4880 - loss: 2.5227
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 348us/step - accuracy: 0.5089 - loss: 2.2382
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - accuracy: 0.5219 - loss: 1.9655
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/s

/home/user/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6601 - loss: 26.2223  
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - accuracy: 0.6223 - loss: 14.9467
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - accuracy: 0.5368 - loss: 9.2150
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.4824 - loss: 6.4946
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - accuracy: 0.4916 - loss: 4.6533
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - accuracy: 0.4929 - loss: 3.6514
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 373us/step - accuracy: 0.4931 - loss: 2.8905
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 359us/step - accuracy: 0.5123 - loss: 2.3404
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 329us/step - accuracy: 0.5309 - loss: 1.9637
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 305us/step - accuracy: 0.5463 - loss: 1.6979
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - accuracy: 0.5759 - loss: 1.4921
Epoch 12/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 332us/st

In [11]:
grid_result.cv_results_

{'mean_fit_time': array([0.82249912, 1.67011309, 2.61888178]),
 'std_fit_time': array([0.0497212 , 0.14252265, 0.0506922 ]),
 'mean_score_time': array([0.08011826, 0.07579144, 0.06992284]),
 'std_score_time': array([0.01791919, 0.01358638, 0.01437485]),
 'param_epochs': masked_array(data=[10, 50, 100],
              mask=[False, False, False],
        fill_value=999999),
 'params': [{'epochs': 10}, {'epochs': 50}, {'epochs': 100}],
 'split0_test_score': array([0.6015625 , 0.67578125, 0.66796875]),
 'split1_test_score': array([0.4609375, 0.59375  , 0.6875   ]),
 'split2_test_score': array([0.60784314, 0.65098039, 0.69411765]),
 'mean_test_score': array([0.55678105, 0.64017055, 0.68319547]),
 'std_test_score': array([0.06782011, 0.03435036, 0.01110069]),
 'rank_test_score': array([3, 2, 1], dtype=int32)}